MOABB to NY format

.CSV to NY

this code convert the data sets in .CSV format to NY format which it can be easily read in Julia and Python. 

The .npz file holds the data and the stimulation vector while the .yml file holds the metadata. 

It has been specifically conceived for BCI data.

This script is for BNCI2014004-test


In [1]:
#import essential libraries

import numpy as np
import pandas as pd
import glob, os, sys, yaml
from yaml import CLoader as Loader, CDumper as Dumper
import moabb
from moabb.paradigms import MotorImagery


In [2]:
# Verify the epochs shape, labels, and metadata of BI2013a if wanted
# Epoch shape >>> trials X channels X time_samples

paradigm = MotorImagery()
dataset = moabb.datasets.BNCI2014004()
subjects = dataset.subject_list

epochs, labels, meta = paradigm.get_data(dataset=dataset, subjects = [subjects[0]], return_epochs = True)


Choosing from all possible events
BNCI2014004 has been renamed to BNCI2014_004. BNCI2014004 will be removed in version 1.1.
The dataset class name 'BNCI2014004' must be an abbreviation of its code 'BNCI2014-004'. See moabb.datasets.base.is_abbrev for more information.


Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
720 matching events found
No baseline correction applied


c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 120 events (all good), 3 – 7.5 s (baseline off), ~3.1 MiB, data loaded,
 'left_hand': 60
 'right_hand': 60>
  warn(f"warnEpochs {epochs}")
c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 120 events (all good), 3 – 7.5 s (baseline off), ~3.1 MiB, data loaded,
 'left_hand': 60
 'right_hand': 60>
  warn(f"warnEpochs {epochs}")
c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 160 events (all good), 3 – 7.5 s (baseline off), ~4.1 MiB, data loaded,
 'left_hand': 80
 'right_hand': 80>
  warn(f"warnEpochs {epochs}")
c:\Users\doumif\.conda\envs\py310\lib\site-packages\moabb\datasets\preprocessing.py:278: UserWarning: warnEpochs <Epochs | 160 events (all good), 3 – 7.5 s (baseline off), ~4.1 MiB, data loaded,
 'left_hand': 80
 '

In [5]:
event_labels = dataset.event_id
event_labels 

{'left_hand': 1, 'right_hand': 2}

In [6]:
epochs.ch_names

['C3', 'Cz', 'C4']

In [3]:
# Forward your file with all .csv of the dataset

file_dir = "C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test"
all_files = [os.path.join(file_dir, file) for file in os.listdir(file_dir)]
all_files #check if you have .csv files

['C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_01_session_03.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_01_session_04.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_01_session_05.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_02_session_03.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_02_session_04.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_02_session_05.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_03_session_03.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_03_session_04.csv',
 'C:\\Users\\doumif\\work\\OfficeWork\\BCI Databases\\CSV\\MI\\BNCI2014004-test\\subject_03_session_05.csv',
 'C:\\Users\\doumif

In [ ]:
#Calc windowlength
samplingrate = 250 #available in dataset description
windowlength = np.diff(dataset.interval)[0]*samplingrate
windowlength

#Calc offset
offset = dataset.interval[0] * samplingrate
print(offset

1125.0

In [5]:
offset = dataset.interval[0] * samplingrate
offset

750

In [6]:
#YML creator 
#The Meta data of the Data set
#These variables and parameters should address once for each data set.  

#######################################Acquisition#######################################
filter= 'Bandpass [0.5-100 Hz], Notch 50 Hz'
ground='Right Mastoid'
hardware= ' Easycap, Germany ; biosignal amplifier (g.tec, Guger Technologies OEG, Graz, Austria)'
reference= 'Left Mastoid'
samplingrate= 250
sensors = epochs.ch_names 
sensortype ='Ag/AgCl Wet electrodes'
software= 'BioSig toolbox (Octave/MATLAB/C++) + SIMULINK'

##############Documentation######################################
description= 'https://lampx.tugraz.at/~bci/database/004-2014/description.pdf'
doi= 'https://doi.org/10.1109/TNSRE.2007.906956'
investigators = "R. Leeb, C. Brunner, G. R. Muller-Putz, A. Schlogl, and G.Pfurtscheller"
place = "Graz University of Technology, Institute for Knowledge Discovery, Laboratory of Brain-Computer Interfaces, Graz, Austria"
repository= 'https://bnci-horizon-2020.eu/database/data-sets'
##id
condition= 'Test'
database= 'BNCI2014004'  
paradigm= 'MI'
timestamp= 2008

##########################################Stim###########################################
####Labels
event_labels = dict(left_hand = 1, right_hand = 2) 
nclasses= len(event_labels.keys())
offset = 750
windowlength = 1125

#########################################Subjects########################################  
subject_ids = dataset.subject_list 
subjects=len(subject_ids)
sessions = 3 
runs = 1 

#function of YMLcreator
def YMLcreator():


  d = dict(acquisition=dict
          (filter= str(filter),
          ground=str(ground),
          hardware=str(hardware),
          reference=str(reference),
          samplingrate= samplingrate,
          sensors= sensors,
          sensortype=str(sensortype),
          software=str(software)
          )
          ,documentation=dict
          (description=str(description),
          doi=str(doi),
          investigators=str(investigators),
          place=str(place),
          repository=str(repository)
          )              
          ,
          formatversion=str('0.0.1')
          ,
          id=dict
          (condition=str(condition),
          database=str(database),
          paradigm=str(paradigm),
          subject=subject,
          session=session,
          run=run,
          timestamp=timestamp,
          )
          ,stim=dict
          (labels=event_labels,
          nclasses=nclasses,
          offset=offset,
          windowlength=windowlength,
          )
          
  )
  return d  

i=-1 #don't change i

for s_index, subject in enumerate(subject_ids):
    for session in range(3, sessions + 3):
        i = i + 1
        filepath = all_files[i]
        for run in range(1, runs + 1):
            d = YMLcreator()
            newpath = os.path.splitext(filepath)[0] + '.yml'
            with open(newpath, 'w') as file:
              documents = yaml.dump(d, file)

              file.write("""
##############################################################################
#                     GIPSA-lab standard for EEG time series (version 0.0.1) #
#                                Authors : Pedro Rodrigues and Marco Congedo #
#                                                        November 15th, 2019 #
##############################################################################

# This format has been conceived for easily sharing EEG data in Python and 
# Julia. Each file is understood as a separate recording. Data consist of two
# files. They have the same name and extensions `npz` and `yml` (this file).

# The `npz` file typically holds the EEG data matrix, a real matrix of 
# dimension num. of samples x num. of electrodes and a vector of integer with
# the tags for stimulations, with as many entries as number of samples. The 
# tags are 0 (zero) for no stimulation and then employs the natural numbers 
# (1, 2,...) for different stimulation classes.

# The `yml` file holds all meta-data info of the recording in `yml` format. 
# It holds two fields and four dictionaries:

# FIELDS:
#
# - paradigm: (string) the experimental paradigm, e.g., P300, MI, ... 
#             for Brain Computer Interfaces experiments
#
# - formatversion: (version) version of this metadata specification

# DICTIONARIES:
#
# - acquisition: (dictionary)
#
#   - filter: (string) filter setting of the EEG acquisition machine, 
#             specifying the type and specification. Ex: "Band-pass digital 
#             filter (0.01-70Hz)"
#   - ground: (string) location of the sensor used as ground. Ex: "Fpz"
#   - reference: (string) location of the sensor used as reference for the 
#                recording. Ex: "A1"
#   - hardware: (string) the commercial name and producer of the the EEG 
#               acquisition machine. Ex: "actiCHamp, Brain Products GmbH 
#               (Germany), DC amplifiers"
#   - software: (string) software used for acquiring and storing the data. 
#               Ex: OpenViBE, INRIA (France)
#   - samplingrate:(int) sampling rate. Ex: 128
#   - sensors: (array-like of strings) location of the sensors, excluding 
#              ground and reference.
#   - sensortype: (string) type, material and product name of electrodes. 
#                 Ex: Ag/AgCl, Braincap, Brain Products GmbH (Germany)
#
# - documentation: (dictionary)
#
#   - description: (string) link to a file or website describing the dataset
#   - doi: (string) digital object identifier of the dataset's documentation
#   - repository:  (string) link to the online repository where the data can 
#                  be downloaded
#
# - id: (dictionary)
#   
#   - database: (string) name of the database
#
# - stim: (dictionary)
#
#   - labels: (dictionary) dictionary with the labels and code of the 
#             stimulations
#       - nclasses: (int) number of classes for the stimulations
#       - offset: (int) offset, given in number of samples, with respect to 
#                 stimulation samples, defining the beginning of trials
#       - windowlength: (int) size of the window, given in number of sample, 
#                       defining the duration of trials
""")

In [11]:

##NPZ creator

#the Structure of the Data sets 
"""
The data should be in the form of  Samples*Channels, where channels may include a column for the time,
a column for the ground, a colum for the trigger, and a colum for the target. So, you need to drop the
columns of time and ground and add the two columns of trigger and target together(If you have not already done so).
"""

timestamp_col = 0  
stim_col= 4 # Stim col

#NPZ creator

def csv2npz(filepath):

    df= pd.read_csv(filepath, header=0) # header = 0 to remove header
    df= np.array(df)
    
    # from V to µV
    raw_data = df[:, 1:-1] * 1e6  

    DATA = np.float32(raw_data)
    print(DATA.shape)

    #STIM is the sum of the two trigger and target columns 
    STIM = np.int16(df[:,stim_col])
    
    unique_values, counts = np.unique(STIM, return_counts=True)
    for value, count in zip(unique_values, counts):
        print(f"Dans {filepath} : Nombre de {value} = {count}")
    
    newpath=os.path.splitext(filepath)[0] + '.npz'
    np.savez(newpath,data=DATA , stim=STIM)

for f in all_files:
    csv2npz(f)

(468810, 3)
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_03.csv : Nombre de 0 = 468650
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_03.csv : Nombre de 1 = 80
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_03.csv : Nombre de 2 = 80
(466123, 3)
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_04.csv : Nombre de 0 = 465963
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_04.csv : Nombre de 1 = 80
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_04.csv : Nombre de 2 = 80
(474354, 3)
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_05.csv : Nombre de 0 = 474194
Dans C:\Users\doumif\work\OfficeWork\BCI Databases\CSV\MI\BNCI2014004-test\subject_01_session_05.csv : Nombre de